In [26]:
import pyaudio
import wave
import whisper
import time
import warnings
warnings.filterwarnings('ignore')
import threading

In [37]:
class System:
    def __init__(self):
        self.audio = pyaudio.PyAudio()
        self.stream = self.audio.open(format=pyaudio.paInt16, channels=1, rate=44100, input=True, frames_per_buffer=1024)
        self.model = whisper.load_model("base.en")
        self.duration = 1
        self.filename = 'audio.wav'
        self.text = ""
        self.result_arr = []
        self.frames = []
        
    def record_audio(self):
        start_time = time.time()
        
        while (time.time() - start_time) < self.duration:
            data = self.stream.read(1024)
            self.frames.append(data)

        with wave.open(self.filename, 'wb') as wf:
            wf.setnchannels(1)
            wf.setsampwidth(self.audio.get_sample_size(pyaudio.paInt16))
            wf.setframerate(44100)
            wf.writeframes(b''.join(self.frames))
            
    def transcribe_audio(self):
        result = self.model.transcribe(self.filename)
        #print(result["text"])
        self.result_arr.append(result["text"])
        if "." in result["text"]:
            self.duration = 1
        else:
            self.duration+=1
        #print("duration {}".format(self.duration))
    
    def check_result(self):
        self.text = self.result_arr[len(self.result_arr)-1]
        print(self.text, end="\r")
            
    def record_and_transcribe(self):
        try:
            while True:
                self.record_audio()
                self.transcribe_audio()
                self.check_result()
        
        except KeyboardInterrupt:
            pass

        self.stream.stop_stream()
        self.stream.close()
        self.audio.terminate()



In [38]:
system = System()
system.record_and_transcribe()